&nbsp;
&nbsp;
![](../_resources/images/e2e-5.png)


# Generative AI with Databricks

## From Predictive to Prescriptive Maintenance
Manufacturers face labor shortages, supply chain disruptions, and rising costs, making efficient maintenance essential. Despite investments in maintenance programs, many struggle to boost asset productivity due to technician shortages and poor knowledge-sharing systems. This leads to knowledge loss and operational inefficiencies.

<div style="font-family: 'DM Sans';">
  <div style="width: 400px; color: #1b3139; margin-left: 50px; margin-right: 50px; float: left;">
    <div style="color: #ff5f46; font-size:50px;">73%</div>
    <div style="font-size:25px; margin-top: -20px; line-height: 30px;">
      of manufacturers struggle to recruit maintenance technicians — McKinsey (2023)
    </div>
    <div style="color: #ff5f46; font-size:50px;">55%</div>
    <div style="font-size:25px; margin-top: -20px; line-height: 30px;">
      of manufacturers lack formal knowledge-sharing systems — McKinsey (2023)
    </div>
  </div>
</div>

Generative AI can transform maintenance by reducing downtime and improving productivity. While predictive maintenance anticipates failures, Generative AI enables prescriptive maintenance. Using historical data, AI systems can identify issues, generate solutions, and assist technicians, allowing junior staff to perform effectively and freeing experts for complex tasks.
<br><br>

### From Models to Agent Systems
Generative AI is moving from standalone models to modular agent systems ([Zaharia et al., 2024](https://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems/)). These systems integrate retrievers, models, prompts, and tools to handle complex tasks. Their modular design allows seamless upgrades (e.g., integrating a new LLM) and adaptation to changing needs.

<br><br>
<img style="float: right; margin-top: 10px;" width="700px" src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/team_flow_liza.png" />


<br>
<div style="font-size: 19px; margin-left: 0px; clear: left; padding-top: 10px; ">
<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" style="float: left;" width="80px"> 
<h3 style="padding: 10px 0px 0px 5px;">Liza, a Generative AI engineer, uses the Databricks Intelligence Platform to:</h3>
<ul style="list-style: none; padding: 0; margin-left: 05%;">
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">1</div>
    Build real-time data pipelines
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">2</div>
    Retrieve vectors & features
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">3</div>
    Create AI agent tools
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">4</div>
    Build & deploy agents
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">5</div>
    Operate in batch or real-time
  </li>
  <li style="display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">6</div>
    Evaluate agent performance
  </li>
</ul>
</div>

**Databricks empowers Liza with a Data + AI platform for Prescriptive Maintenance.** Let’s explore how to deploy this in production.

<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=4003492105941350&notebook=%2F05-Generative-AI%2F05.1-ai-tools-iot-turbine-prescriptive-maintenance&demo_name=lakehouse-iot-platform&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-iot-platform%2F05-Generative-AI%2F05.1-ai-tools-iot-turbine-prescriptive-maintenance&version=1">

## Building Agent Systems with Databricks Mosaic AI agent framework

We will build an Agent System designed to generate prescriptive work orders for wind turbine maintenance technicians. This system integrates multiple interacting components to ensure proactive and efficient maintenance, thereby optimizing the overall equipment effectiveness.

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/iot_agent_graph_v2_0.png" style="margin-left: 5px; float: right"  width="1000px;">

Databricks simplifies this by providing a built-in service to:

- Create and store your AI tools leveraging UC functions
- Execute the AI tools in a safe way
- Use agents to reason about the tools you selected and chain them together to properly answer your question. 


This notebook creates the three Mosaic AI tools and associated Mosaic AI endpoints, which will be composed together into a agent in notebook [05.2-agent-creation-guide]($./05.2-agent-creation-guide).
1. **Turbine predictor** which uses a Model Serving endpoint to predict turbines at risk of failure.
2. **Turbine specifications retriever** which retrieve the turbine specifications based on its id.
3. **Turbine maintenance guide**  which uses a Vector Search endpoint to retrieve maintenance guide based on the turbines and issues being adressed.

In [0]:
%pip install databricks-sdk==0.40.0 
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../_resources/00-setup $reset_all_data=false

## Configuration file

Please change your catalog and schema here to run the demo on a different catalog.

 
<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=4003492105941350&notebook=%2Fconfig&demo_name=lakehouse-iot-platform&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-iot-platform%2Fconfig&version=1">


# Technical Setup notebook. Hide this cell results
Initialize dataset to the current user and cleanup data when reset_all_data is set to true

Do not edit

USE CATALOG `main`
using catalog.database `main`.`dbdemos_iot_turbine`


data already existing. Run with reset_all_data=true to force a data cleanup for your local demo.


In [0]:
%sql
DROP FUNCTION IF EXISTS turbine_specifications_retriever;

--turbine_specifications_retriever to get the current status of a turbine
--This function is used to retrieve the turbine specifications based on its id

CREATE OR REPLACE FUNCTION 
turbine_specifications_retriever(turbine_id STRING COMMENT 'ID of the wind turbine to look up')
RETURNS TABLE (
  avg_energy DOUBLE COMMENT 'Average energy reading',
  std_sensor_A DOUBLE COMMENT 'Sensor A reading',
  std_sensor_B DOUBLE COMMENT 'Sensor B reading',
  std_sensor_C DOUBLE COMMENT 'Sensor C reading',
  std_sensor_D DOUBLE COMMENT 'Sensor D reading',
  std_sensor_E DOUBLE COMMENT 'Sensor E reading',
  std_sensor_F DOUBLE COMMENT 'Sensor F reading'
)
LANGUAGE SQL
COMMENT 'This function retrieves the turbine sensor readings / specifications based on the turbine_id'
RETURN
(
SELECT 

avg_energy, std_sensor_A, std_sensor_B, std_sensor_C, std_sensor_D, std_sensor_E, std_sensor_F
FROM main.dbdemos_iot_turbine.turbine_current_features
WHERE turbine_id = turbine_specifications_retriever.turbine_id
SORT BY hourly_timestamp DESC
limit 1
);

In [0]:
%sql
SELECT * FROM turbine_specifications_retriever('004a641f-e9e5-9fff-d421-1bf88319420b')

